In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk

In [3]:
# training of the model through pipelines
df = pd.read_csv("C:/Users/Shahbaz Baig/Desktop/Titanic-Dataset.csv")

In [4]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
203,204,0,3,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C
128,129,1,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
802,803,1,1,"Carter, Master. William Thornton II",male,11.0,1,2,113760,120.0000,B96 B98,S
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C
266,267,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,3101295,39.6875,NaN,S


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest ,chi2


In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.drop(['Name','Ticket','Cabin','PassengerId'],axis= 1,inplace = True)

In [8]:
df


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [9]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(['Survived'],axis=1),
                                                  df['Survived'],
                                                  random_state = 102 , 
                                                    test_size = 0.2)

In [63]:
x_train


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
190,2,female,32.00,0,0,13.000,S
456,1,male,65.00,0,0,26.550,S
853,1,female,16.00,0,1,39.400,S
374,3,female,3.00,3,1,21.075,S
333,3,male,16.00,2,0,18.000,S
...,...,...,...,...,...,...,...
626,2,male,57.00,0,0,12.350,Q
242,2,male,29.00,0,0,10.500,S
590,3,male,35.00,0,0,7.125,S
755,2,male,0.67,1,1,14.500,S


In [45]:
# you pass list of tuples in column transformer
#imputation Transformer 
from sklearn.impute import SimpleImputer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),['Age']),
    ('impute_Embarked' , SimpleImputer(strategy = 'most_frequent'),['Embarked'])
    
],remainder= 'passthrough') #if i dont write passthrough then my remaining columns will automatically dropped

In [79]:
trf2 = ColumnTransformer(
    transformers=[
        ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
    ],
    remainder='passthrough'
)

In [83]:
trf2
print(x_train[:5])

     Pclass     Sex   Age  SibSp  Parch    Fare Embarked
190       2  female  32.0      0      0  13.000        S
456       1    male  65.0      0      0  26.550        S
853       1  female  16.0      0      1  39.400        S
374       3  female   3.0      3      1  21.075        S
333       3    male  16.0      2      0  18.000        S


In [35]:
trf3 = ColumnTransformer([
    ('scale' ,MinMaxScaler(),slice(0,10))
])

In [37]:
# this is for feature selection and k = 5 means you are using top 5 columns for training
trf4 = SelectKBest(score_func=chi2,k=8)

In [33]:
trf5 = DecisionTreeClassifier()

In [39]:
pipe = Pipeline([
    ('trf1' , trf1),
    ('trf2' , trf2),
    ('trf3' , trf3),
    ('trf4' , trf4)
]
)

In [41]:
# alternate syntax for pipeline
pipe2 = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [ ]:
pipe.fit(x_train,y_train)

In [55]:
print(trf2.fit_transform(x_train))



[[ 1.     0.     0.    ...  0.     0.    13.   ]
 [ 0.     1.     0.    ...  0.     0.    26.55 ]
 [ 1.     0.     0.    ...  0.     1.    39.4  ]
 ...
 [ 0.     1.     0.    ...  0.     0.     7.125]
 [ 0.     1.     0.    ...  1.     1.    14.5  ]
 [ 1.     0.     1.    ...  0.     0.    79.2  ]]


In [65]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=['ohe_sex_embarked',
                                                 OneHotEncoder(handle_unknown='ignore',
                                                               sparse_output=False),
                                                 [1, 6]])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000025AE3CFCC20>))])

In [71]:
print(type(x_train))

<class 'pandas.core.frame.DataFrame'>


In [73]:
x_train.shape

(712, 7)